In [1]:
import gc
import pandas as pd
import glob
import sklearn
import numpy as np

In [2]:
diFiles=glob.glob("../Di/*")
diFiles

['../Di/Jnk.csv',
 '../Di/IRF4.csv',
 '../Di/LEF1.csv',
 '../Di/Ikk2.csv',
 '../Di/CTNNB1.csv',
 '../Di/Erk.csv',
 '../Di/MYC.csv']

In [3]:
trainIndexFiles=glob.glob("../Train/*")
frames = list()
for f in trainIndexFiles:
    indexes = pd.read_csv(f,sep=";")
    indexList=list(indexes['0'])
    fpath = '../Di/'+f.split("/")[-1].split(".")[0]+".csv"
    dcRead = pd.read_csv(fpath,sep=";")
    dcPartialTrain=dcRead[dcRead.columns[5:]].loc[indexList].copy()
    del dcRead
    gc.collect()
    frames.append(dcPartialTrain)

In [4]:
diTrain = pd.concat(frames)

In [7]:
diTrain.to_csv("../FullTrain/FullTrainDI/FullTrainDI.csv")

In [8]:
lstn_clusters=range(2,30)

In [9]:
import time
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score

In [10]:
file = open('../ClusteringOutputs/DI_Clustering/diMiniBatch.txt','w')  
bestOut = open('../ClusteringOutputs/DI_Clustering/diBestMiniBatch.txt','w') 
file.write("n_clusters,s_score\n")

19

In [ ]:
max_silhouette=0
nClusters=2
t0 = time.time()

for n in lstn_clusters:
    mbk=MiniBatchKMeans(init='k-means++', n_clusters=n
                        , max_no_improvement=10, verbose=0)

    mbk.fit(diTrain)
    labels = mbk.labels_
    s_score = silhouette_score(diTrain, labels, metric='euclidean')
    
    file.write(str(n)+",")
    file.write(str(s_score)+"\n")
    if s_score>max_silhouette:
        max_silhouette = s_score
        nClusters=n
t_mini_batch = time.time() - t0    
#print("Num clusters " + str(n))
bestOut.write("Num clusters " + str(n))
#print("Silhouette score " + str(max_silhouette))
bestOut.write("Silhouette score " + str(max_silhouette))
#print("Running time " + str(t_mini_batch))
file.close()
bestOut.close()